In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [13]:

dados = pd.read_csv('matchinfo.csv')

In [14]:
dados.head(1)

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,blueTop,...,redTopChamp,redJungle,redJungleChamp,redMiddle,redMiddleChamp,redADC,redADCChamp,redSupport,redSupportChamp,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,Dyrus,...,Gnar,Meteos,Elise,Hai,Fizz,Sneaky,Sivir,LemonNation,Thresh,http://matchhistory.na.leagueoflegends.com/en/...


In [15]:
dados.columns

Index(['League', 'Year', 'Season', 'Type', 'blueTeamTag', 'bResult', 'rResult',
       'redTeamTag', 'gamelength', 'blueTop', 'blueTopChamp', 'blueJungle',
       'blueJungleChamp', 'blueMiddle', 'blueMiddleChamp', 'blueADC',
       'blueADCChamp', 'blueSupport', 'blueSupportChamp', 'redTop',
       'redTopChamp', 'redJungle', 'redJungleChamp', 'redMiddle',
       'redMiddleChamp', 'redADC', 'redADCChamp', 'redSupport',
       'redSupportChamp', 'Address'],
      dtype='object')

In [21]:
dados['redTopChamp'].value_counts()


Maokai          880
Gnar            665
Shen            604
Rumble          526
Nautilus        396
Poppy           384
Renekton        326
Trundle         324
Fiora           284
JarvanIV        232
Irelia          229
Ekko            229
Gragas          226
Gangplank       213
Camille         166
Kennen          149
Kled            133
Hecarim         128
Chogath         115
Jayce           111
Fizz            109
Lissandra        98
Jax              97
Lulu             83
Malphite         76
Galio            75
Ryze             69
DrMundo          52
TahmKench        51
Sion             51
               ... 
Aatrox            6
Pantheon          5
Nasus             5
Urgot             5
Zac               5
Mordekaiser       4
RekSai            4
Rengar            4
Zed               3
Yorick            3
Teemo             2
Talon             2
LeeSin            2
Malzahar          2
Taliyah           2
Warwick           2
Karma             2
Azir              2
Khazix            2


In [22]:
dados['blueTopChamp'].value_counts()

Maokai         827
Shen           657
Gnar           625
Rumble         571
Nautilus       400
Poppy          386
Trundle        329
Renekton       307
Fiora          247
JarvanIV       237
Camille        221
Gragas         216
Irelia         214
Gangplank      212
Ekko           205
Kennen         160
Jayce          146
Hecarim        135
Chogath        123
Kled           116
Ryze           113
Jax            111
Lulu            99
Fizz            97
Galio           88
Lissandra       83
Malphite        71
Sion            59
TahmKench       55
Swain           47
              ... 
Sejuani          7
Akali            5
Yorick           5
Alistar          5
RekSai           4
Nidalee          4
Azir             4
Pantheon         3
Nasus            3
Malzahar         3
LeeSin           3
Mordekaiser      2
Talon            2
Rengar           2
Nocturne         2
Teemo            2
Zac              2
Urgot            2
Nunu             1
Warwick          1
XinZhao          1
Taliyah     

In [26]:
#porcentagem de wins per champ no top
top = {}
for i in dados.blueTopChamp:
    top[i]=0
for i in dados.redTopChamp:
    top[i]=0
for k,v in  top.items():
    wins=len(dados[(dados.blueTopChamp==k) & (dados.bResult==1)])+len(dados[(dados.redTopChamp==k) & (dados.rResult==1)])
    matchs=len(dados[dados.blueTopChamp==k])+ len(dados[dados.redTopChamp==k])      
    if matchs!=0:
        top[k]=wins/matchs
print(top)

{'Irelia': 0.4966139954853273, 'Gnar': 0.5, 'Renekton': 0.4597156398104265, 'Kassadin': 0.5555555555555556, 'Sion': 0.5454545454545454, 'Jax': 0.4807692307692308, 'Lulu': 0.45054945054945056, 'Maokai': 0.5125951962507322, 'Rumble': 0.5050136736554239, 'Lissandra': 0.48066298342541436, 'RekSai': 0.25, 'Fizz': 0.5436893203883495, 'Morgana': 0.5263157894736842, 'Kennen': 0.4336569579288026, 'Gragas': 0.45701357466063347, 'DrMundo': 0.27586206896551724, 'Vladimir': 0.43529411764705883, 'Hecarim': 0.4600760456273764, 'Ryze': 0.5, 'Ekko': 0.5184331797235023, 'Shen': 0.4964314036478985, 'Nautilus': 0.5012562814070352, 'Khazix': 0.3333333333333333, 'Riven': 0.5, 'Olaf': 0.4473684210526316, 'Shyvana': 0.47368421052631576, 'Yasuo': 0.5517241379310345, 'Malphite': 0.47619047619047616, 'Fiora': 0.5009416195856874, 'TahmKench': 0.5471698113207547, 'Poppy': 0.4883116883116883, 'Gangplank': 0.52, 'Graves': 0.47540983606557374, 'Quinn': 0.5, 'Trundle': 0.5252679938744257, 'Pantheon': 0.75, 'Rammus': 0

In [28]:
#porcentagem de wins per team
dic={}
for i in dados.blueTeamTag:
    dic[i]=0
for i in dados.redTeamTag:
    dic[i]=0
for k,v in dic.items():
    wins=len(dados[(dados.blueTeamTag==k) & (dados.bResult==1)])+len(dados[(dados.redTeamTag==k) & (dados.rResult==1)])
    matchs=len(dados[dados.blueTeamTag==k])+ len(dados[dados.redTeamTag==k])
    if matchs!=0:
        dic[k]=wins/matchs
print (dic)

{'TSM': 0.6509433962264151, 'CST': 0.375, 'WFX': 0.36363636363636365, 'TIP': 0.5, 'CLG': 0.559322033898305, 'DIG': 0.4388888888888889, 'TL': 0.47509578544061304, 'C9': 0.5836065573770491, 'T8': 0.44680851063829785, 'GV': 0.52, 'TDK': 0.2903225806451613, 'NME': 0.24, 'REN': 0.4090909090909091, 'FOX': 0.34375, 'IMT': 0.6197916666666666, 'NRG': 0.3188405797101449, 'NV': 0.4069264069264069, 'APX': 0.4716981132075472, 'P1': 0.463855421686747, 'FLY': 0.45045045045045046, 'EL': 0.37037037037037035, 'GIA': 0.41361256544502617, 'CW': 0.3617021276595745, 'SK': 0.5, 'GMB': 0.5555555555555556, 'MYM': 0.2631578947368421, 'ROC': 0.42021276595744683, 'FNC': 0.5765124555160143, 'UOL': 0.5095785440613027, 'H2K': 0.6124031007751938, 'OG': 0.4382716049382716, 'SPY': 0.5096774193548387, 'VIT': 0.4426229508196721, 'G2': 0.6327433628318584, 'S04': 0.43548387096774194, 'MSF': 0.5181818181818182, 'NIP': 0.28205128205128205, 'MM': 0.19444444444444445, 'CJE': 0.4528301886792453, 'JAG': 0.47735191637630664, 'SKT